### LoRA Fine-tune optimization - RoBERTa Token Classification

#### Hardware Spec CPU/GPU
CPU       : Intel(R) Xeon(R) CPU E5-2698 v4 @ 2.20GHz 20 core(40thread) * 2 ea</br>
Memory : 512GB</br>
DISK      : /  ( 440 GB)</br>
             /raid (7 TB)</br>    
GPU : nvidia V100 16GB * 8 ea</br>

In [47]:
!nvidia-smi

Thu Aug  8 09:12:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:06:00.0 Off |                    0 |
| N/A   41C    P0              61W / 300W |   7197MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

|   1  Tesla V100-SXM2-16GB           On  | 00000000:07:00.0 Off |                    0 |
| N/A   43C    P0              60W / 300W |   7635MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   2  Tesla V100-SXM2-16GB           On  | 00000000:0A:00.0 Off |                    0 |
| N/A   42C    P0              60W / 300W |   7455MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   3  Tesla V100-SXM2-16GB           On  | 00000000:0B:00.0 Off |                    0 |
| N/A   40C    P0              58W / 300W |   7431MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+---------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


|   7  Tesla V100-SXM2-16GB           On  | 00000000:8A:00.0 Off |                    0 |
| N/A   38C    P0              57W / 300W |   7111MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A   3081712      C   /usr/bin/python3                           7192MiB |
|    1   N

In [3]:
!pip install -q peft transformers datasets evaluate seqeval

In [37]:
from datasets import load_dataset
from transformers import (
    AutoModelForSeq2SeqLM, 
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    DataCollatorForTokenClassification, 
    TrainingArguments, 
    Trainer
)
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig
import evaluate
import torch
import numpy as np

In [65]:
## Hyperparameters
model_checkpoint = 'roberta-base'
lr = 1e-3
batch_size = 16
# num_epochs = 1
num_epochs = 10

[codecarbon INFO @ 09:24:34] Energy consumed for RAM : 0.530869 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:24:34] Energy consumed for all GPUs : 1.354630 kWh. Total GPU Power : 475.16304777434163 W
[codecarbon INFO @ 09:24:34] Energy consumed for all CPUs : 0.119523 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:34] 2.005022 kWh of electricity used since the beginning.


In [6]:
# !huggingface-cli login
# https://huggingface.co/settings/tokens
# Login with token

In [48]:
# tags = label_id
from datasets import load_dataset

bionlp = load_dataset('tner/bionlp2004')
# bionlp['train'][0]
print(bionlp)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 16619
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1927
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3856
    })
})


[codecarbon INFO @ 09:17:49] Energy consumed for RAM : 0.509634 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:17:49] Energy consumed for all GPUs : 1.294504 kWh. Total GPU Power : 474.78588229298134 W
[codecarbon INFO @ 09:17:49] Energy consumed for all CPUs : 0.114742 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:17:49] 1.918880 kWh of electricity used since the beginning.


In [49]:
bionlp['train'][0]

{'tokens': ['Since',
  'HUVECs',
  'released',
  'superoxide',
  'anions',
  'in',
  'response',
  'to',
  'TNF',
  ',',
  'and',
  'H2O2',
  'induces',
  'VCAM-1',
  ',',
  'PDTC',
  'may',
  'act',
  'as',
  'a',
  'radical',
  'scavenger',
  '.'],
 'tags': [0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [50]:
label_dict = {
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10
}

In [51]:
label_list = [
    "O",
    "B-DNA",
    "I-DNA",
    "B-protein",
    "I-protein",
    "B-cell_type",
    "I-cell_type",
    "B-cell_line",
    "I-cell_line",
    "B-RNA",
    "I-RNA",
]

In [52]:
# precision, accuracy, F1, and recall 등 sequence labeling tasks evaluation module  
seqeval = evaluate.load("seqeval")

In [53]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        'precision': results["overall_precision"],
        'recall': results["overall_recall"],
        'f1': results['overall_f1'],
        'accuracy': results['overall_accuracy'],
    }


In [54]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
# 주로 RoBERTa, GPT-2와 같은 특정 토크나이저에서 사용되며, 텍스트의 각 토큰 앞에 공백을 추가하는 옵션입니다.

In [55]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f'tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i) #토크나이저가 토큰화된 입력에서 원본 텍스트의 각 단어에 해당하는 인덱스를 반환합니다
        previous_word_idx = None # word_ids를 사용하면 각 토큰이 원본 텍스트의 어떤 단어에 해당하는지 추적할 수 있습니다.
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

## Tokenization

In [56]:
tokenized_bionlp = bionlp.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3856 [00:00<?, ? examples/s]

Map: 100%|██████████| 3856/3856 [00:00<00:00, 11564.96 examples/s]


In [57]:
## Collator (padding) in trainer2
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [58]:
id2label = {
    0: "O",
    1: "B-DNA",
    2: "I-DNA",
    3: "B-protein",
    4: "I-protein",
    5: "B-cell_type",
    6: "I-cell_type",
    7: "B-cell_line",
    8: "I-cell_line",
    9: "B-RNA",
    10: "I-RNA",
}
label2id = {
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}

In [59]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=11, id2label=id2label, label2id=label2id
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## PEFT Model configuration

In [60]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias='all'
)

[codecarbon INFO @ 09:18:04] Energy consumed for RAM : 0.510420 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:18:04] Energy consumed for all GPUs : 1.296481 kWh. Total GPU Power : 474.79592726104863 W
[codecarbon INFO @ 09:18:04] Energy consumed for all CPUs : 0.114919 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:18:04] 1.921820 kWh of electricity used since the beginning.


In [61]:
model_peft = get_peft_model(model, peft_config)
model_peft.print_trainable_parameters()

trainable params: 700,427 || all params: 124,661,782 || trainable%: 0.5619


[codecarbon INFO @ 09:18:49] Energy consumed for RAM : 0.512780 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:18:49] Energy consumed for all GPUs : 1.302412 kWh. Total GPU Power : 474.79414259759403 W
[codecarbon INFO @ 09:18:49] Energy consumed for all CPUs : 0.115450 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:18:49] 1.930642 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:19:19] Energy consumed for RAM : 0.514353 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:19:19] Energy consumed for all GPUs : 1.306366 kWh. Total GPU Power : 474.8493369080622 W
[codecarbon INFO @ 09:19:19] Energy consumed for all CPUs : 0.115804 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:19:19] 1.936523 kWh of electricity used since the beginning.


### Training Config

In [ ]:
training_args = TrainingArguments(
    output_dir='roberta-base-lora-token-classification',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    # evaluation_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

[codecarbon INFO @ 09:24:49] Energy consumed for RAM : 0.531655 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:24:49] Energy consumed for all GPUs : 1.356608 kWh. Total GPU Power : 475.08680878046965 W
[codecarbon INFO @ 09:24:49] Energy consumed for all CPUs : 0.119700 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:49] 2.007964 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:25:04] Energy consumed for RAM : 0.532442 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:25:04] Energy consumed for all GPUs : 1.358586 kWh. Total GPU Power : 475.04704857301755 W
[codecarbon INFO @ 09:25:04] Energy consumed for all CPUs : 0.119877 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:25:04] 2.010906 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:25:19] Energy consumed for RAM : 0.533229 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:25:19] Energy consumed for all GPUs : 1.360565 kWh. Total GPU Power : 475.07846027768

### Training - Compare PEFT and Non-PEFT

In [67]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bionlp['train'],
    eval_dataset=tokenized_bionlp['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

[codecarbon INFO @ 09:35:31] [setup] RAM Tracking...
[codecarbon INFO @ 09:35:31] [setup] GPU Tracking...
[codecarbon INFO @ 09:35:31] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:35:31] [setup] CPU Tracking...
[codecarbon WARNING @ 09:35:31] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 09:35:32] We saw that you have a Intel(R) Xeon(R) CPU E5-2698 v4 @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 09:35:32] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2698 v4 @ 2.20GHz
[codecarbon INFO @ 09:35:32] >>> Tracker's metadata:
[codecarbon INFO @ 09:35:32]   Platform system: Linux-5.15.0-1029-nvidia-x86_64-with-glibc2.35
[codecarbon INFO @ 09:35:32]   Python version: 3.10.12
[codecarbon INFO @ 09:35:32]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 09:35:32]   Available RAM : 503.756 GB
[codecarbon INFO @ 09:35:32]   CPU count: 80
[codecarbon INFO @ 09:35:32]   CPU model: Intel(R) Xeon(R) CPU E5-2698 v4 

In [68]:
trainer_peft = Trainer(
    model=model_peft,
    args=training_args,
    train_dataset=tokenized_bionlp['train'],
    eval_dataset=tokenized_bionlp['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

[codecarbon INFO @ 09:35:48] [setup] RAM Tracking...
[codecarbon INFO @ 09:35:48] [setup] GPU Tracking...
[codecarbon INFO @ 09:35:48] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:35:48] [setup] CPU Tracking...
[codecarbon WARNING @ 09:35:48] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 09:35:49] Energy consumed for RAM : 0.566261 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:35:49] Energy consumed for all GPUs : 1.443590 kWh. Total GPU Power : 474.3332895218547 W
[codecarbon INFO @ 09:35:49] Energy consumed for all CPUs : 0.127491 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:35:49] 2.137341 kWh of electricity used since the beginning.
[codecarbon WARNING @ 09:35:49] We saw that you have a Intel(R) Xeon(R) CPU E5-2698 v4 @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 09:35:49] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2698 v4 @ 2.20GHz
[codecarbon INFO @ 09:35:49] >>> Tracker's

[codecarbon INFO @ 09:36:04] Energy consumed for RAM : 0.567047 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:36:04] Energy consumed for all GPUs : 1.445565 kWh. Total GPU Power : 474.38168466092367 W
[codecarbon INFO @ 09:36:04] Energy consumed for all CPUs : 0.127668 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:04] 2.140281 kWh of electricity used since the beginning.


In [70]:
trainer.train()

/home/dgxuser/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/dgxuser/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.158933,0.770387,0.821718,0.795225,0.947072
2,No log,0.161260,0.776978,0.816856,0.796418,0.946738
3,No log,0.153313,0.782403,0.832703,0.806770,0.948981
4,0.140800,0.155910,0.783479,0.824959,0.803684,0.948194
5,0.140800,0.157452,0.778350,0.817036,0.797224,0.948036
6,0.140800,0.157215,0.795216,0.820277,0.807553,0.949571
7,0.140800,0.158701,0.784062,0.823879,0.803477,0.947958
8,0.119400,0.155866,0.784937,0.831442,0.807521,0.949217
9,0.119400,0.156560,0.788988,0.830902,0.809403,0.949945
10,0.119400,0.158031,0.791423,0.827481,0.809050,0.950083


[codecarbon INFO @ 09:46:04] Energy consumed for RAM : 0.598497 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:46:04] Energy consumed for all GPUs : 1.588980 kWh. Total GPU Power : 828.9198782862011 W
[codecarbon INFO @ 09:46:04] Energy consumed for all CPUs : 0.134753 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:46:04] 2.322230 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:46:08] Energy consumed for RAM : 0.000788 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:46:08] Energy consumed for all GPUs : 0.004101 kWh. Total GPU Power : 983.3711972481027 W
[codecarbon INFO @ 09:46:08] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:46:08] 0.005066 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:46:19] Energy consumed for RAM : 0.599283 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:46:19] Energy consumed for all GPUs : 1.593148 kWh. Total GPU Power : 1000.791189097725

TrainOutput(global_step=1300, training_loss=0.12523043705866888, metrics={'train_runtime': 436.351, 'train_samples_per_second': 380.863, 'train_steps_per_second': 2.979, 'total_flos': 9843034024162548.0, 'train_loss': 0.12523043705866888, 'epoch': 10.0})

[codecarbon INFO @ 09:53:19] Energy consumed for RAM : 0.621286 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:53:19] Energy consumed for all GPUs : 1.707592 kWh. Total GPU Power : 572.1401911543174 W
[codecarbon INFO @ 09:53:19] Energy consumed for all CPUs : 0.139892 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:53:19] 2.468770 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:53:34] Energy consumed for RAM : 0.622072 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:53:34] Energy consumed for all GPUs : 1.709802 kWh. Total GPU Power : 530.7189948491502 W
[codecarbon INFO @ 09:53:34] Energy consumed for all CPUs : 0.140069 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:53:34] 2.471943 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:53:49] Energy consumed for RAM : 0.622859 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:53:49] Energy consumed for all GPUs : 1.711999 kWh. Total GPU Power : 527.689266835899 

In [69]:
trainer_peft.train()

/home/dgxuser/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/dgxuser/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.189910,0.710247,0.782640,0.744688,0.935896
2,No log,0.179172,0.731671,0.792545,0.760892,0.939222
3,No log,0.166569,0.749795,0.824599,0.785420,0.944023
4,0.196000,0.158513,0.756964,0.817216,0.785937,0.945360
5,0.196000,0.158041,0.765921,0.807852,0.786328,0.946954
6,0.196000,0.154047,0.782504,0.818296,0.800000,0.948646
7,0.196000,0.156399,0.770809,0.818837,0.794097,0.947308
8,0.149600,0.151609,0.783218,0.835404,0.808470,0.949374
9,0.149600,0.150401,0.780071,0.830362,0.804431,0.949374
10,0.149600,0.151547,0.784458,0.827120,0.805224,0.949512


[codecarbon INFO @ 09:36:19] Energy consumed for RAM : 0.567834 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:36:19] Energy consumed for all GPUs : 1.449097 kWh. Total GPU Power : 848.1817212197427 W
[codecarbon INFO @ 09:36:19] Energy consumed for all CPUs : 0.127845 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:19] 2.144776 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:36:22] Energy consumed for RAM : 0.000795 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:36:22] Energy consumed for all GPUs : 0.004024 kWh. Total GPU Power : 955.0618763093194 W
[codecarbon INFO @ 09:36:22] Energy consumed for all CPUs : 0.000179 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:22] 0.004998 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:36:34] Energy consumed for RAM : 0.568620 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:36:34] Energy consumed for all GPUs : 1.453236 kWh. Total GPU Power : 994.2129266736409

TrainOutput(global_step=1300, training_loss=0.16434309445894682, metrics={'train_runtime': 431.129, 'train_samples_per_second': 385.476, 'train_steps_per_second': 3.015, 'total_flos': 9843034024162548.0, 'train_loss': 0.16434309445894682, 'epoch': 10.0})

[codecarbon INFO @ 09:43:19] Energy consumed for RAM : 0.589846 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:43:19] Energy consumed for all GPUs : 1.565173 kWh. Total GPU Power : 898.4314892502883 W
[codecarbon INFO @ 09:43:19] Energy consumed for all CPUs : 0.132805 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:43:19] 2.287823 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:43:34] Energy consumed for RAM : 0.590632 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:43:34] Energy consumed for all GPUs : 1.567333 kWh. Total GPU Power : 518.9383820400602 W
[codecarbon INFO @ 09:43:34] Energy consumed for all CPUs : 0.132982 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:43:34] 2.290947 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:43:49] Energy consumed for RAM : 0.591418 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:43:49] Energy consumed for all GPUs : 1.569466 kWh. Total GPU Power : 512.0612823899429

## Inference Test

In [42]:
peft_model_id = './roberta-base-lora-token-classification/checkpoint-130'
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForTokenClassification.from_pretrained(
    config.base_model_name_or_path, num_labels=11, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(inference_model, peft_model_id)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[codecarbon INFO @ 09:01:18] Energy consumed for RAM : 0.457728 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:01:18] Energy consumed for all GPUs : 1.164028 kWh. Total GPU Power : 474.7653341742885 W
[codecarbon INFO @ 09:01:18] Energy consumed for all CPUs : 0.103055 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:01:18] 1.724811 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:01:33] Energy consumed for RAM : 0.458514 kWh. RAM Power : 188.9085144996643 W
[codecarbon INFO @ 09:01:33] Energy consumed for all GPUs : 1.166005 kWh. Total GPU Power : 474.795006926234 W
[codecarbon INFO @ 09:01:33] Energy consumed for all CPUs : 0.103232 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:01:33] 1.727752 kWh of electricity used since the beginning.


### Sample text inference

In [43]:
text = "The activation of IL-2 gene expression and NF-kappa B through CD28 requires reactive oxygen production by 5-lipoxygenase."
inputs = tokenizer(text, return_tensors='pt')

In [44]:
with torch.no_grad():
    logits = model(**inputs).logits

tokens = inputs.tokens()
predictions = torch.argmax(logits, dim=2)

for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))


('<s>', 'O')
('The', 'O')
('Ġactivation', 'O')
('Ġof', 'O')
('ĠIL', 'B-DNA')
('-', 'I-DNA')
('2', 'I-DNA')
('Ġgene', 'I-DNA')
('Ġexpression', 'O')
('Ġand', 'O')
('ĠNF', 'B-protein')
('-', 'I-protein')
('k', 'I-protein')
('appa', 'I-protein')
('ĠB', 'I-protein')
('Ġthrough', 'O')
('ĠCD', 'B-protein')
('28', 'I-protein')
('Ġrequires', 'O')
('Ġreactive', 'O')
('Ġoxygen', 'O')
('Ġproduction', 'O')
('Ġby', 'O')
('Ġ5', 'B-protein')
('-', 'O')
('lip', 'O')
('oxy', 'O')
('gen', 'O')
('ase', 'O')
('.', 'O')
('</s>', 'O')


In [45]:
## Ġrequires - Ġ is the blank at the RoBERTa model.